## Rodando localmente

### Carregando o modelo

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Pega o nome do modelo que está lá no HaggingFace
model_name = 'Qwen/Qwen2-1.5B-Instruct' #gpt2 para testar modelos mais leves
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Utilizado pipeline para geração de texto

In [4]:
from transformers import pipeline

gerador = pipeline('text-generation', model=model, tokenizer=tokenizer)
resultado = gerador('Era uma vez', max_length=25) #Pedindo para completar o texto com até 25 tokens
print(resultado)

Device set to use cpu


[{'generated_text': 'Era uma vez um pequeno reino chamado Cemil, que era famoso por sua beleza e r'}]


In [5]:
resultado[0]['generated_text']

'Era uma vez um pequeno reino chamado Cemil, que era famoso por sua beleza e r'

### Gerando texto manualmente

In [6]:
input_ids = tokenizer.encode('Era uma vez', return_tensors='pt')
input_ids

tensor([[   36,   956, 10608, 20563]])

In [7]:
output = model.generate(input_ids, max_length=25)
output

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tensor([[   36,   956, 10608, 20563,  4443, 80672, 11790, 41665, 33709,  2123,
           619,   567,   318,    13,   506,   281, 21715, 11385, 15886,   321,
           384,  1079, 43315,    11,  9243]])

In [10]:
resultado = tokenizer.decode(output[0], skip_special_tokens=True)
print(resultado)

Era uma vez um pequeno país chamado Jardim. O povo era gentil e amável, mas


### Criando uma estrutura de conversação

In [11]:
messages = [{'role':'user', 'content':'Olá'}] # user fala Olá
text = tokenizer.apply_chat_template(messages,tokenize=False, add_generation_prompt=True)
inputs = tokenizer([text], return_tensors='pt')
inputs

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  42719,   1953, 151645,    198,
         151644,  77091,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
outputs = model.generate(inputs.input_ids, max_length=25, do_sample=True)
resultado = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(resultado)

system
You are a helpful assistant.
user
Olá
assistant
Olá! Como


#### Criando um pequeno loop de conversa

In [ ]:
mensagens = []
for i in range(3):
    messages.append({'role':'user', 'content':input('Fale algo com o assitente: ')})
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors='pt')
    outputs = model.generate(inputs.input_ids, max_length=500, do_sample=True)
    resultado = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta_assist = resultado.split('assistant')[-1].strip('/n')
    print(resposta_assist)
    messages.append({'role':'assistant', 'content':resposta_assist})